In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import lightgbm as lgb
from dask_ml.model_selection import train_test_split, GridSearchCV
from dask.distributed import Client
from sklearn.metrics import roc_auc_score
import gc  # 가비지 컬렉션 모듈

In [2]:
client = Client()

In [3]:
# 데이터 불러오기 및 전처리 함수
def preprocess_data(file_path):
    df = dd.read_csv(file_path)
    df = df.drop('ID', axis=1)
    
    fill_zero_columns = ['F04', 'F11', 'F18', 'F19', 'F24', 'F27', 'F29', 'F32', 'F33', 'F36', 'F38']
    df[fill_zero_columns] = df[fill_zero_columns].fillna(0)
    df = df.fillna('NAN')

In [4]:
 # 데이터 타입 최적화
float_columns = df.select_dtypes(include=['float64']).columns
df[float_columns] = df[float_columns].astype('float32')
    
int_columns = df.select_dtypes(include=['int64']).columns
df[int_columns] = df[int_columns].astype('int32')
    
    object_columns = df.select_dtypes(include=['object']).columns
    df[object_columns] = df[object_columns].astype('category')
    
    return df

IndentationError: unexpected indent (1524448177.py, line 2)

In [ ]:
# Dask로 데이터 불러오기 및 전처리
df = preprocess_data('train.csv')

In [ ]:
# 데이터 분할
X = df.drop('Click', axis=1)
y = df['Click']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 모델 정의 및 하이퍼파라미터 튜닝
model = lgb.LGBMClassifier(objective='binary', n_jobs=-1)
param_grid = {
    'num_leaves': [31, 63],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200]
}

In [ ]:
# GridSearchCV 설정
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='roc_auc', verbose=1)
grid_search.fit(X_train.compute(), y_train.compute())

In [ ]:
# 최적 모델 선정
best_model = grid_search.best_estimator_

In [ ]:
# 검증 데이터에서의 성능 평가
y_valid_pred = best_model.predict_proba(X_valid)[:, 1]
print('Validation AUC:', roc_auc_score(y_valid, y_valid_pred))

In [ ]:
# 메모리 청소
del df, X_train, X_valid, y_train, y_valid, X, y
gc.collect()

In [ ]:
# 테스트 데이터 로드 및 전처리
test_df = preprocess_data('test.csv')

In [ ]:
# 예측
pred = best_model.predict_proba(test_df.compute())[:, 1]

In [ ]:
# 제출 파일 생성
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['Click'] = pred
sample_submission.to_csv('lgbm_optimized.csv', index=False)